# Copy data elements from one instance to another
Do some basic things with metadata using the DHIS2 API

In [ ]:
# Load up environment
from dotenv import load_dotenv
import os

load_dotenv()

print('HANDY_DHIS2_USER_SOURCE: {}'.format(os.getenv('HANDY_DHIS2_USER_SOURCE')))
print('HANDY_DHIS2_PASSWORD_SOURCE: {}'.format(os.getenv('HANDY_DHIS2_PASSWORD_SOURCE')))
print('HANDY_DHIS2_HOST_SOURCE: {}'.format(os.getenv('HANDY_DHIS2_HOST_SOURCE')))

print('HANDY_DHIS2_USER_DEST: {}'.format(os.getenv('HANDY_DHIS2_USER_DEST')))
print('HANDY_DHIS2_PASSWORD_DEST: {}'.format(os.getenv('HANDY_DHIS2_PASSWORD_DEST')))
print('HANDY_DHIS2_HOST_DEST: {}'.format(os.getenv('HANDY_DHIS2_HOST_DEST')))

In [ ]:
import requests
import json

try:
    r = requests.get(os.getenv('HANDY_DHIS2_HOST_SOURCE')+'/api/dataElements.json?paging=false&fields=*',
              auth=(os.getenv('HANDY_DHIS2_USER_SOURCE'),os.getenv('HANDY_DHIS2_PASSWORD_SOURCE'))  )
except requests.exceptions.RequestException as e:  # This is the correct syntax
    print(e)

response = r.json()

with open('dataelments.json', 'w') as outfile:
    json.dump(response, outfile)


In [ ]:
# Another way to filter the data elements by exact name

names = [
'Covid-19 Aztrazeneca 1 M',
'Covid-19 Pfizer 1 M',
'Covid-19 Pfizer 1 F',
'Covid-19 Sinorpharm1 M',
'Covid-19 Sinorpharm1 F',
'Covid-19 Johnson&Jonshon M',
'Covid-19 Johnson&Jonshon F',
'Covid-19 Aztrazeneca 2 M',
'Covid-19 Aztrazeneca 2 F',
'Covid-19 Pfizer 2 M',
'Covid-19 Pfizer 2 F',
'Covid-19 Sinorpharm 2 M',
'Covid-19 Sinorpharm 2 F',
'Covid-19 Aztrazeneca 3 M',
'Covid-19 Aztrazeneca 3 F',
'Covid-19 Pfizer 3 M',
'Covid-19 Pfizer 3 F',
'Covid-19 Sinorpharm 3 M',
'Covid-19 Sinorpharm 3 F',
'Covid-19 Johnson&Jonshon 2 M',
'Covid-19 Johnson&Jonshon 2 F',
'Total Enfants sains suivis durant le mois Nouveau (F) 0-5 mois',
'Total Enfants sains suivis durant le mois Nouveau (F) 6-11 mois',
'Total Enfants sains suivis durant le mois Nouveau (F) 12-23 mois',
'Total Enfants sains suivis durant le mois Nouveau (F) 24-35 mois',
'Total Enfants sains suivis durant le mois Nouveau (F) 36-59 mois',
'Total Enfants sains suivis durant le mois Nouveau (M) 0-5 mois',
'Total Enfants sains suivis durant le mois Nouveau (M) 6-11 mois',
'Total Enfants sains suivis durant le mois Nouveau (M) 12-23 mois',
'Total Enfants sains suivis durant le mois Nouveau (M) 24-35 mois',
'Total Enfants sains suivis durant le mois Nouveau (F) 0-5 mois',
'Total Enfants sains suivis durant le mois Nouveau (F) 6-11 mois',
'Total Enfants sains suivis durant le mois Nouveau (F) 12-23 mois',
'Total Enfants sains suivis durant le mois Nouveau (M) 0-5 mois',
'Total Enfants sains suivis durant le mois Nouveau (M) 6-11 mois',
'Total Enfants sains suivis durant le mois Nouveau (M) 12-23 mois',
'R20_Total consultations Activités curatives',
'R20_POP_Ensemble',
'Palu:ACT adulte-Traitement Dispensé',
'Palu:ACT petit enfant-Traitement Dispensé',
'Palu:ACT grand enfant-Traitement Dispensé',
'Palu:ACT NRSS-Traitement Dispensé',
]

In [ ]:
import json

# Now load this in
f = open('dataelments.json')
data = json.load(f)


dataelements = []
for n in data['dataElements']:
    dataelement = {}

    if n['name'] in names:
        dataelement['id'] = n['id']
        dataelement['name'] = n['name']
        dataelement['shortName'] = n['shortName']
        dataelement['aggregationType'] = n['aggregationType']
        dataelement['domainType'] = n['domainType']
        dataelement['valueType'] = n['valueType']
        dataelements.append(dataelement)
    
# put it in an object
data = {}
data['dataElements'] = dataelements
print('found {}'.format( len(dataelements) ) + ' of {}'.format( len(names)) +' data elements')

In [ ]:
# write out (opttional)
with open('file_for_importing.json', 'w') as outfile:
    json.dump(data, outfile, ensure_ascii=False, indent=4)


In [ ]:
import requests

# and load this up
try:
    r = requests.post(os.getenv('HANDY_DHIS2_HOST_DEST')+'/api/metadata.json', json=data,
              auth=(os.getenv('HANDY_DHIS2_USER_DEST'),os.getenv('HANDY_DHIS2_PASSWORD_DEST'))  )
except requests.exceptions.RequestException as e:  # This is the correct syntax
    print(e)

r.text

In [ ]:
# Here is the curl command if you want
print('curl --user '+os.getenv('HANDY_DHIS2_USER_DEST')+':'+os.getenv('HANDY_DHIS2_PASSWORD_DEST')+' '+os.getenv('HANDY_DHIS2_HOST_DEST') + '/api/metadata.json --header "Content-Type: application/json" --data @file_for_importing.json')